In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('fr-en.pair.tsv',sep = '\t',on_bad_lines='skip')
data.head()

/var/folders/xr/ww1bx5dx03ncctbbx4zptmcm0000gn/T/ipykernel_5593/3455895076.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('fr-en.pair.tsv',sep = '\t',on_bad_lines='skip')


,4.,4..1,ep-09-11-26-008-04.txt,008-04,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Ratification et mise en œuvre des conventions ...,The ratification and implementation of the upd...,ep-09-11-26-008-04.txt,008-04,NaN,NaN,NaN,NaN
1,7.,7.,ep-08-03-12-005-07.txt,005-07,NaN,NaN,NaN,NaN
2,Agriculture durable et biogaz: nécessité de re...,Sustainable agriculture and biogas: review of ...,ep-08-03-12-005-07.txt,005-07,NaN,NaN,NaN,NaN
3,- Avant le vote sur le paragraphe 41:,- Before the vote on paragraph 41:,ep-08-03-12-005-07.txt,005-07,NaN,NaN,NaN,NaN
4,"rapporteur. - (EN) Monsieur le Président, nous...","rapporteur. - Mr President, we agreed to chang...",ep-08-03-12-005-07.txt,005-07,101,Csaba Sándor Tabajdi,NaN,NaN


In [2]:
data=data.rename(columns={'4.':'inputs','4..1':'targets'})
data=data[['inputs','targets']]
data['index']=data.index
data.set_index('index',inplace=True)

In [3]:
data.head()

,inputs,targets
index,,
0,Ratification et mise en œuvre des conventions ...,The ratification and implementation of the upd...
1,7.,7.
2,Agriculture durable et biogaz: nécessité de re...,Sustainable agriculture and biogas: review of ...
3,- Avant le vote sur le paragraphe 41:,- Before the vote on paragraph 41:
4,"rapporteur. - (EN) Monsieur le Président, nous...","rapporteur. - Mr President, we agreed to chang..."


In [4]:
train_data, val_data = train_test_split(data, test_size=0.2)

In [5]:
from transformers import MarianTokenizer
model_token = 'Helsinki-NLP/opus-mt-fr-en'
tokenizer = MarianTokenizer.from_pretrained(model_token,return_tensors = 'pt')

/Applications/anaconda3/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
max_length=128
def preprocess_function(row):
    inputs = row['inputs']
    targets = row['targets']
    if not isinstance(inputs,(str,list)):
        inputs = str(inputs)
    if not isinstance(targets,(str,list)):
        targets = str(targets)
    model_inputs=tokenizer(inputs,text_target=targets,max_length=max_length,truncation=True)
    return model_inputs

In [9]:
tokenized_datasets = train_data.apply(preprocess_function,axis=1)


In [10]:
test_datasets = val_data.apply(preprocess_function,axis=1)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_token)

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model = model)

In [ ]:
import evaluate
metric = evaluate.load('sacrebleu')

In [ ]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(tokenized_datasets,shuffle =True,collate_fn = data_collator,batch_size=8)
eval_dataloader=DataLoader(test_datasets,collate_fn = data_collator,batch_size=8)

In [122]:
train_dataloader

In [124]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_token)

In [125]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/.venv/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [126]:
from accelerate import Accelerator
accelerator = Accelerator()
model,optimizer,train_dataloader,eval_dataloader =accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [127]:
from transformers import get_scheduler
num_train_epochs =3
num_steps = len(train_dataloader)
num_train_steps = num_train_epochs * num_steps

lr_scheduler = get_scheduler('linear', optimizer =optimizer,num_warmup_steps =0,num_training_steps=num_train_steps)

In [128]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [129]:
output_dir = './saved_model/'

In [130]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_train_steps))


for epoch in range(num_train_epochs):
    
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)


  0%|          | 0/572988 [00:00<?, ?it/s]

KeyError: 848036